# What is Explainable AI?

**Explainable AI** is an emerging field in machine learning that aims to address how black box decisions of AI are made. This area inspects and tries to understand the steps & models involved in making decisions. It answers questions like - 

1. Why did the AI system make a specific prediction or decision?
2. Why didn’t the AI system do something else?
3. When do AI system give enough confidence in the decision that one can trust it, how can the AI system correct    errors that arise?

To promote explainable AI, researchers have been developing tools and techniques such as What-if Tool, LIME,SHAP TreeInterpreter and many more.



# Shapley value

A prediction can be explained by assuming that each feature value of the instance is a "player" in a game where the prediction is the payout. Shapley values -- a method from coalitional game theory -- tells us how to fairly distribute the "payout" among the features.

Lloyd shapley Idea:- Members should recieve payments or share propotionals to their marginal contribution.

The Shapley value is the only attribution method that satisfies the properties Efficiency, Symmetry, Dummy and Additivity, which together can be considered a definition of a fair payout.

To get deeper into the definition and computation of the Shapley value , you can refer this link https://christophm.github.io/interpretable-ml-book/shapley.html



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')

In [ ]:
df.head(10)

In [ ]:
output=['toxic','severe_toxic','obscene','threat','insult','identity_hate']
res_df1=df.loc[(df['toxic'] == 1) | (df['severe_toxic'] == 1) | (df['obscene'] == 1) | (df['threat'] == 1) | (df['insult'] == 1) | (df['identity_hate'] == 1)] 
res_df1.loc[:,'label'] = 1
res_df2=df.loc[(df['toxic'] == 0) & (df['severe_toxic'] == 0) & (df['obscene'] == 0) & (df['threat'] == 0) & (df['insult'] == 0) & (df['identity_hate'] == 0)] 
res_df2.loc[:,'label'] = 0
df=res_df1.append(res_df2)
df = df.sample(frac=1).reset_index(drop=True)
df.head(5)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['comment_text'],df['label'], test_size=0.3, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf= TfidfVectorizer(min_df=2,max_df=0.5,ngram_range=(1,1),max_features=10000,stop_words='english')
features= tfidf.fit_transform(X_train)



In [ ]:
features=features.todense()
test_tfidf= tfidf.transform(X_test)
test_tfidf=test_tfidf.todense()

In [ ]:
#random
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=15, n_estimators=40)
clf.fit(features, y_train)

In [ ]:
test_data=X_test.tolist()
test_data[12445]

# Working with Shap Library

Here we use Force Plot to visualize the contribution of various features(words) which makes the sentence as toxic or non toxic

In [ ]:
import shap
explainer = shap.TreeExplainer(clf)
choosen_instance = test_tfidf[12445]
choosen_instance = np.squeeze(np.asarray(choosen_instance))
shap_values = explainer.shap_values(choosen_instance)
shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[0],choosen_instance,feature_names=tfidf.get_feature_names())

Here you can see the contribution of **thanks** word is quite high as compare to other features

In [ ]:
test_data[37]

In [ ]:
choosen_instance = test_tfidf[37]
choosen_instance = np.squeeze(np.asarray(choosen_instance))
shap_values = explainer.shap_values(choosen_instance)
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1],choosen_instance,feature_names=tfidf.get_feature_names())

Here Words like fuck, balls , ass have contribution which makes a toxic sentence.

In [ ]:
test_data[75]

In [ ]:
choosen_instance = test_tfidf[75]
choosen_instance = np.squeeze(np.asarray(choosen_instance))
shap_values = explainer.shap_values(choosen_instance)
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1],choosen_instance,feature_names=tfidf.get_feature_names())

another toxic example